**ABDSynth demo notebook**

This notebook allows the user to test the ABDSynth pre-trained model on a sample patient from the AMOS MR dataset. 

Notes: 
- Make sure to create an appropriate virtual environment according to https://github.com/BBillot/SynthSeg
- You will need sufficient GPU memory to run this notebook, ~40GB. 

Deepa Krishnaswamy and Cosmin Ciausu
Brigham and Women's Hospital
May 2025

If you haven't already, clone the SynthSeg repository. 

In [1]:
!git clone https://github.com/BBillot/SynthSeg.git

Cloning into 'SynthSeg'...
remote: Enumerating objects: 2479, done.
remote: Counting objects: 100% (683/683), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 2479 (delta 613), reused 538 (delta 538), pack-reused 1796 (from 1)
Receiving objects: 100% (2479/2479), 120.57 MiB | 64.61 MiB/s, done.
Resolving deltas: 100% (1614/1614), done.


In [ ]:
import os
import sys
import nibabel as nib 
import numpy as np 

curr_path = os.getcwd()
sys.path.insert(0, os.path.join(curr_path, 'SynthSeg'))

In [8]:
from SynthSeg.predict import predict as synthseg_predict

Using TensorFlow backend.


Then we download the model weights from github release attachments. 

In [21]:
!curl -f -L -O https://github.com/deepakri201/ABDSynth/releases/download/v1.0.0/dice_100.h5 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  151M  100  151M    0     0  57.4M      0  0:00:02  0:00:02 --:--:-- 72.8M


In [24]:
# inputs 
model_weights_filename = os.path.join(curr_path, 'dice_100.h5')
path_images = os.path.join(curr_path, 'data')
path_images_filename = os.path.join(curr_path, 'data', 'amos_0540.nii.gz')
labels_segmentation_npy_filename = os.path.join(curr_path, 'npy', 'synthseg_segmentation_labels.npy')
# output folder 
path_segmentations = os.path.join(curr_path, 'output')
path_segmentations_filename = os.path.join(curr_path, 'output', 'amos_0540_synthseg.nii.gz')

assert(os.path.exists(model_weights_filename))
assert(os.path.exists(path_images))
assert(os.path.exists(path_images_filename))
assert(os.path.exists(labels_segmentation_npy_filename))

In [25]:
# predict 
synthseg_predict(path_images=path_images_filename,
                path_segmentations=path_segmentations_filename,
                path_model=model_weights_filename,
                labels_segmentation=labels_segmentation_npy_filename,
                target_res=1.5,
                gradients=False,
                flip=False,
                sigma_smoothing=0,
                keep_biggest_component=True,
                n_levels = 5,
                nb_conv_per_level = 2,
                conv_size = 3,
                unet_feat_count = 24,
                activation = 'elu',
                feat_multiplier = 2,
                verbose=True)

predicting 1/1


The predicted segmentation has isotropic spacing of 1.5mm. So we need to resample it back to the original spacing of the volume. 

We can also use itkwidgets to overlay the segmentations on the image. 